In [ ]:
from datetime import datetime

import numpy as np
import sys
import soundfile as sf
import torch

sys.path.insert(0, '../')
from editor_app.schemas import UtteranceSTTCreate
from editor_app.stt import transcribe_utterance
from editor_app.database import SessionLocal
from editor_app.crud import get_project_by_title, create_utterance_stt, get_utterances_stt
from utils import compute_string_similarity
from IPython.display import display, Audio

In [ ]:
db = SessionLocal()

In [ ]:
title = 'morts_insolites_89'
user_id = 1
project = get_project_by_title(db, title, user_id)

scores = []
for utter in project.utterances:
    stt_text, lang = transcribe_utterance(utter)
    levenstein_score = compute_string_similarity(utter.text, stt_text)
    score = round(levenstein_score, 3)
    print(score, utter.text, stt_text)
    
    utter_stt = UtteranceSTTCreate(orig_utterance_id = utter.id,
                                   text=stt_text,
                                   levenstein_similarity=round(levenstein_score,3),
                                   date=datetime.now())
    create_utterance_stt(db, utter_stt)
    scores.append(score)
    
np.mean(scores)

### Reduction
Text enclosed in square brakets will be reducted with the help of wav2vecalligner. this allows to handle issues with repetiotions automatically.

In [ ]:
from tortoise.utils.wav2vec_alignment import Wav2VecAlignment

aligner = Wav2VecAlignment()

In [ ]:
print(project.utterances[4].text)
wav_path = project.utterances[4].get_audio_path()
wav, sample_rate = sf.read(wav_path)
wav = torch.from_numpy(wav).unsqueeze(0).to(dtype=torch.float32, device='cuda')
# expected_text = 'His beginnings already suggested a destiny of star. In May 1963, Keith Rell founded the famous rock band, The Yardbirds, with guitarist Anthony Toffum, his high school friend. [With his baritone voice and his high school friend.] With his baritone voice and his harmonica flights, Keith captured all the light.'
expected_text = 'His beginnings already suggested a destiny of star. In May 1963, Keith Rell founded the famous rock band, The Yardbirds, with guitarist Anthony Toffum, his high school friend. With his baritone voice and his harmonica flights, Keith captured all the light.'
res_wav = aligner.redact(wav, expected_text)
Audio(data=res_wav.cpu(), rate=sample_rate)

In [ ]:
path = project.utterances[4].get_audio_path()
path.unlink()
sf.write(path, res_wav.cpu().squeeze(0), sample_rate)

In [ ]:
def compute_project_mean_score(db, title, user_id):

    project = get_project_by_title(db, title, user_id)
    scores = []
    sort_func = lambda x: x.date
    for utter in project.utterances:
        assert len(utter.utterance_stt) > 0
        utterance_stt = sorted(utter.utterance_stt, key=sort_func)[-1]
        scores.append(utterance_stt.levenstein_similarity)
    return sum(scores) / len(scores)

In [ ]:
compute_project_mean_score(db, title, user_id)